In [11]:
import polars as pl

In [12]:
pitching = pl.read_csv("./data/lahman_1871-2024_csv/pitching.csv")
pitching.head()

playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,BAOpp,ERA,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""aardsda01""",2004,1,"""SFN""","""NL""",1,0,11,0,0,0,0,32,20,8,1,10,5,0.417,6.75,0,0,2,0,61,5,8,0,1,1
"""aardsda01""",2006,1,"""CHN""","""NL""",3,0,45,0,0,0,0,159,41,24,9,28,49,0.214,4.08,0,1,1,0,225,9,25,1,3,2
"""aardsda01""",2007,1,"""CHA""","""AL""",2,1,25,0,0,0,0,97,39,23,4,17,36,0.3,6.4,3,2,1,0,151,7,24,2,1,1
"""aardsda01""",2008,1,"""BOS""","""AL""",4,2,47,0,0,0,0,146,49,30,4,35,49,0.268,5.55,2,3,5,0,228,7,32,3,2,4
"""aardsda01""",2009,1,"""SEA""","""AL""",3,6,73,0,0,0,38,214,49,20,4,34,80,0.19,2.52,3,2,0,0,296,53,23,2,1,2


In [13]:

league_ERA = (
    pitching
    .filter(pl.col("yearID") >= 1946)
    .group_by(["yearID", "lgID"])
    .agg([
        pl.col("ER").sum().alias("total_ER"),
        pl.col("IPouts").sum().alias("total_IPouts")
    ])
    .with_columns(
        ((pl.col("total_ER") * 9) / (pl.col("total_IPouts") / 3)).alias("league_ERA")
    )
)
league_ERA.head()

yearID,lgID,total_ER,total_IPouts,league_ERA
i64,str,i64,i64,f64
2009,"""AL""",10006,60520,4.464012
2023,"""NL""",10476,64529,4.383332
1950,"""NL""",5066,33029,4.14127
1958,"""NL""",4841,33077,3.951598
1964,"""AL""",5887,43845,3.625248


In [22]:
print("Current league_ERA structure:")
print(league_ERA.head())
print(f"Columns: {league_ERA.columns}")
def assign_dh_state(year, league):
    if year < 1973:
    return "No DH (Both Leagues)"
    elif year >= 2022:
        return "Universal DH"
    elif league == "AL":
         return "AL with DH"
    elif league == "NL":
        return "NL without DH"
    else:
        return "Unknown"
league_ERA_with_DH = league_ERA.with_columns(
    pl.struct(["yearID", "lgID"]).map_batches(lambda x: [assign_dh_state(row["yearID"], row["lgID"]) for row in x]).alias("DH_state")
)

print("\nLeague ERA with DH states:")
print(league_ERA_with_DH.head(10))

try:
    league_ERA_with_DH_v2 = league_ERA.with_columns(
        pl.when(pl.col("yearID") < 1973).then(pl.lit("No DH (Both Leagues)"))
        .otherwise(
        pl.when(pl.col("yearID") >= 2022)
            .then(pl.lit("Universal DH"))
            .otherwise(
                pl.when(pl.col("lgID") == "AL")
                .then(pl.lit("AL with DH"))
                .otherwise(pl.lit("NL without DH"))
            )
        )
        .alias("DH_state")
    )
    
    print("\nMethod 2 worked! Using nested when-then:")
    print(league_ERA_with_DH_v2.head(10))
    league_ERA_with_DH = league_ERA_with_DH_v2
    
except Exception as e:
    print(f"\nMethod 2 failed: {e}")
    print("Using Method 1 result")
print("\nUnique DH states:")
print(league_ERA_with_DH.select("DH_state").unique().sort("DH_state"))
league_ERA_with_DH.write_csv("./data/league_ERA_analysis.csv")
print("\nData saved to ./data/league_ERA_analysis.csv")
print("\nSample of saved data (sorted by year and league):")
print(league_ERA_with_DH.sort(["yearID", "lgID"]).head(15))

Current league_ERA structure:
shape: (5, 5)
┌────────┬──────┬──────────┬──────────────┬────────────┐
│ yearID ┆ lgID ┆ total_ER ┆ total_IPouts ┆ league_ERA │
│ ---    ┆ ---  ┆ ---      ┆ ---          ┆ ---        │
│ i64    ┆ str  ┆ i64      ┆ i64          ┆ f64        │
╞════════╪══════╪══════════╪══════════════╪════════════╡
│ 2009   ┆ AL   ┆ 10006    ┆ 60520        ┆ 4.464012   │
│ 2023   ┆ NL   ┆ 10476    ┆ 64529        ┆ 4.383332   │
│ 1950   ┆ NL   ┆ 5066     ┆ 33029        ┆ 4.14127    │
│ 1958   ┆ NL   ┆ 4841     ┆ 33077        ┆ 3.951598   │
│ 1964   ┆ AL   ┆ 5887     ┆ 43845        ┆ 3.625248   │
└────────┴──────┴──────────┴──────────────┴────────────┘
Columns: ['yearID', 'lgID', 'total_ER', 'total_IPouts', 'league_ERA']

League ERA with DH states:
shape: (10, 6)
┌────────┬──────┬──────────┬──────────────┬────────────┬─────────────────────────────────┐
│ yearID ┆ lgID ┆ total_ER ┆ total_IPouts ┆ league_ERA ┆ DH_state                        │
│ ---    ┆ ---  ┆ ---      ┆ ---  